In [2]:
from openai import OpenAI
import ollama
MODEL_LLAMA = 'gemma2:2b'
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [3]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [4]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [5]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [6]:
def optimize_gpt(python):    
    stream = ollama_via_openai.chat.completions.create(model=MODEL_LLAMA, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
exec(pi)

Result: 3.146567747183
Execution Time: 0.000000 seconds


In [15]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>
#include <thread>

using namespace std;

void calculate(int iterations, double param1, double param2) {
    double result = 1.0; 
    for (int i = 1; i <= iterations + 1; ++i) {
        double j = i * param1 - param2;
        result -= (1 / j);
        j = i * param1 + param2;
        result += (1 / j);
    }
    cout << result << " ";  
}

int main() {
    double iterations = 100;    
    double param1 = 4.0;    
    double param2 = 1.0; 

    // Time performance measurement 
    auto start = chrono::high_resolution_clock::now();  
    calculate(iterations, param1, param2); 
    auto end = chrono::high_resolution_clock::now();   
    auto duration = chrono::duration_cast<chrono::milliseconds>(end - start).count();

    cout << "Result: " << setprecision(12) << fixed << calculate(iterations, param1, param2); 
    cout << " Execution Time: [" << duration << " ms]" << endl;   
}



```